<a href="https://colab.research.google.com/github/hiroporn/moji-okoshi-srt/blob/main/moji_okoshi_srt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title GoogleDriveをマウントする
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#@title データを入れるフォルダ作成
import os

# Add folders
checkContentFolder = os.path.exists("content")
checkDownLoadFolder = os.path.exists("download")
if not checkContentFolder:
    os.mkdir("content")
if not checkDownLoadFolder:
    os.mkdir("download")

# マウントしたGoogleドライブから音声データをcontentに移動する

In [ ]:
#@title Whisperとsrtの実装

!pip install srt

!pip install git+https://github.com/openai/whisper.git

In [ ]:
#@title 実際に文字起こしを実行する
# 文字起こしされたものは、srtファイルで出力する

import srt
import whisper

from srt import Subtitle
from datetime import timedelta

# ファイル名は右側で入れられるように小細工
file_name = "3-3.mp3"#@param {type:"string"}

# モデルをsmallにするとCPUの場合で、ほぼ等倍程度で出力されるっぽい？
# GPUが使える場合は、largeとかのほうが精度が良さそう。
model = whisper.load_model("medium")

# 録音データに無音時間が無い場合は、tenperatureの設定0.8とか。min=0.7をしても良いかも？
# 現状で設定していないのは、長めの暫時休憩があったデータで、同じ単語を繰り返して崩壊したため
# result = model.transcribe(f"content/{file_name}", language="ja",  temperature=0.8)

result = model.transcribe(f"content/{file_name}",
                          language="ja",
                          verbose=False,
                          without_timestamps=False,
                          )

# 文字を30文字程度で折返す設定。どっかからのコピペ
def add_line(s):
    new_s = s
    s_count = len(s)
    s_max_count = 40
    if s_count >= s_max_count:
        if (s_count - s_max_count) >= 3:
            # 30文字以上、かつ、2行目が3文字以上あれば、改行する
            # つまり、43文字以上であれば、40文字で改行する
            new_s = s[:s_max_count] + "\n" + s[s_max_count:]
 
    return new_s

segments = result["segments"]

subs = []
 
for data in segments:
    index = data["id"] + 1
    start = data["start"]
    end = data["end"]
    text = add_line(data["text"])
    sub = Subtitle(index=1, start=timedelta(seconds=timedelta(seconds=start).seconds, microseconds=timedelta(seconds=start).microseconds), 
                   end=timedelta(seconds=timedelta(seconds=end).seconds, microseconds=timedelta(seconds=end).microseconds), content=text, proprietary='')
    subs.append(sub)
 
with open(f'download/{file_name}' + ".srt", mode="w", encoding="utf-8") as f:
    f.write(srt.compose(subs))


100%|██████████████████████████████████████| 1.42G/1.42G [00:10<00:00, 148MiB/s]
100%|██████████| 377434/377434 [09:15<00:00, 679.53frames/s]
